In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms import transforms
from torchsummary import summary
from torch import optim
from torch.utils.data import DataLoader,random_split
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
from torchvision import models

In [3]:
t=transforms.Compose([  transforms.ToTensor(),
                        transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),
                        transforms.RandomHorizontalFlip(0.5),
                        transforms.RandomRotation(10),
                        ])

In [7]:
dataset=ImageFolder(root=r"./data/plantVillage/PlantVillage",transform=t)
train_data,val_data=random_split(dataset,[0.8,0.2])

In [8]:
train_loader=DataLoader(train_data,batch_size=32,shuffle=True)
test_loader=DataLoader(val_data,batch_size=32,shuffle=False)


In [ ]:
model=models.resnet18()
summary(model,input_size=(3,256,256))
model

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           9,408
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
         MaxPool2d-4           [-1, 64, 16, 16]               0
            Conv2d-5           [-1, 64, 16, 16]          36,864
       BatchNorm2d-6           [-1, 64, 16, 16]             128
              ReLU-7           [-1, 64, 16, 16]               0
            Conv2d-8           [-1, 64, 16, 16]          36,864
       BatchNorm2d-9           [-1, 64, 16, 16]             128
             ReLU-10           [-1, 64, 16, 16]               0
       BasicBlock-11           [-1, 64, 16, 16]               0
           Conv2d-12           [-1, 64, 16, 16]          36,864
      BatchNorm2d-13           [-1, 64, 16, 16]             128
             ReLU-14           [-1, 64,

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [13]:
for param in model.parameters():
    param.requires_grad=False
    
in_features=model.fc.in_features
model.fc=nn.Linear(512,15)
for param in model.parameters():
    param.requires_grad=True

In [14]:
criterian=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.fc.parameters(),lr=0.0001)

In [ ]:
num_epoch=10
for epoch in range(num_epoch):
    running_loss=0.0
    for image,label in train_loader:
        output=model(image)
        loss=criterian(output,label)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        running_loss+=loss.item()
    print(f"epoch[{epoch+1}/{num_epoch}]- loss:{running_loss/len(train_loader)}")
    model.eval()
    with torch.no_grad():
        correct=0.0
        total=0.0
        for image,label in test_loader:
            output=model(image)
            _,predicted=torch.max(output,1)
            correct+=(predicted==label).sum().item()
            total+=label.size(0) 
        print(f"accuracy: {(correct/total)*100}")
        torch.save(model.state_dict(),"model.pth")

KeyboardInterrupt: 

In [ ]:
model.eval()
    with torch.no_grad():
        correct=0.0
        total=0.0
        for image,label in test_loader:
            output=model(image)
            _,predicted=torch.max(output,1)
            correct+=(predicted==label).sum().item()
            total+=label.size(0) 
        print(f"accuracy: {(correct/total)*100}")
        torch.save(model.state_dict(),"model.pth")